<a href="https://colab.research.google.com/github/A01723072/Sebasti-n-C-rdenas/blob/main/Equipo1_Calculadora_Plan_de_Retiro_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install streamlit
import streamlit as st
import pandas as pd

# --- Funciones de cálculo ---

def calculate_future_value(initial_investment, annual_contribution, annual_rate_of_return, num_years):
    """Calcula el valor futuro de una inversión."""
    future_value = initial_investment * (1 + annual_rate_of_return)**num_years
    for year in range(1, num_years + 1):
        future_value += annual_contribution * (1 + annual_rate_of_return)**(num_years - year)
    return future_value

def calculate_present_value(desired_annual_income, annual_inflation_rate, years_until_retirement, years_in_retirement):
    """Calcula el valor presente de las necesidades futuras de ingresos."""
    present_value = 0
    for year in range(years_in_retirement):
        present_value += desired_annual_income * (1 + annual_inflation_rate)**year / (1 + 0.05)**(years_until_retirement + year)
    return present_value

def calcular_aportación_mensual_requerida(aportación_inicial, tasa_rendimiento_anual, años_hasta_jubilación, ingreso_anual_deseado, tasa_inflación_anual, años_en_jubilación):
    """Calcula la tasa de ahorro mensual requerida para cumplir los objetivos de jubilación."""
    valor_futuro_inversiones = calculate_future_value(aportación_inicial, 0, tasa_rendimiento_anual, años_hasta_jubilación)
    valor_presente_ingresos = calculate_present_value(ingreso_anual_deseado, tasa_inflación_anual, años_hasta_jubilación, años_en_jubilación)
    aportación_mensual_requerida = (valor_presente_ingresos - valor_futuro_inversiones) / (((1 + tasa_rendimiento_anual)**años_hasta_jubilación - 1) / tasa_rendimiento_anual)
    return aportación_mensual_requerida

def obtener_tasa_rendimiento_anual(años_hasta_jubilación):
    """Devuelve la tasa de rendimiento anual según los años hasta la jubilación."""
    if años_hasta_jubilación >= 10:
        return 0.12  # IPC (12% anual) para inversiones a largo plazo
    else:
        return 0.0867  # CETTES (8.67% anual) para inversiones a corto plazo

# --- Interfaz de usuario con Streamlit ---

# Configurar el título con un estilo personalizado
st.markdown("""
    <style>
        .title {
            font-family: 'Arial Black', sans-serif;
            font-size: 48px;
            color: #007bff; /* Color azul */
            text-align: center;
        }
    </style>
    <div class="title">Calculadora de Jubilación</div>
""", unsafe_allow_html=True)

# Entradas del usuario
edad_actual = st.number_input("Introduce tu edad actual:", min_value=0, value=30)
edad_jubilación = st.number_input("Introduce la edad a la que deseas jubilarte:", min_value=0, value=65)
esperanza_vida = st.number_input("Introduce tu esperanza de vida:", min_value=0, value=85)
ingreso_mensual_deseado = st.number_input("Introduce el ingreso mensual deseado durante la jubilación:", min_value=0.0, value=50000.0)
aportación_inicial = st.number_input("Introduce la cantidad de tu aportación inicial:", min_value=0.0, value=100000.0)


# --- Cálculos ---

# Calcula el número de años hasta la jubilación y los años en jubilación
años_hasta_jubilación = edad_jubilación - edad_actual
años_en_jubilación = esperanza_vida - edad_jubilación

# --- Obtener tasa dinámica y calcular la aportación requerida ---
tasa_rendimiento_anual = obtener_tasa_rendimiento_anual(años_hasta_jubilación)
aportación_mensual_requerida = calcular_aportación_mensual_requerida(aportación_inicial, tasa_rendimiento_anual, años_hasta_jubilación, ingreso_mensual_deseado * 12, 0.03, años_en_jubilación)

# --- Mostrar resultados con estilos ---

# Resultado de la aportación mensual
st.markdown(f"""
    <style>
        .result {{
            font-family: 'Courier New', monospace;
            font-size: 24px;
            color: #28a745; /* Color verde */
            text-align: center;
        }}
    </style>
    <div class="result">Tu aportación mensual estimada requerida es: **{aportación_mensual_requerida:.2f}**</div>
""", unsafe_allow_html=True)


# --- Distribución de la inversión y cálculo del monto total acumulado ---

# Define tasas de rendimiento para renta variable y renta fija
tasa_renta_variable = 0.12  # IPC (12% anual)
tasa_renta_fija = 0.0867  # CETTES (8.67% anual)

# Distribución de la inversión según los años hasta la jubilación
if años_hasta_jubilación >= 10:
    # Mayor proporción en renta variable para inversiones a largo plazo
    porcentaje_renta_variable = 0.70
    porcentaje_renta_fija = 0.30
    mensaje_distribucion = "**Perfil de Inversión:** Agresivo (70% Renta Variable, 30% Renta Fija). **Fundamento:** Al tener un horizonte de inversión a largo plazo, se busca maximizar el crecimiento del capital asumiendo un mayor riesgo."
else:
    # Mayor proporción en renta fija para inversiones a corto plazo
    porcentaje_renta_variable = 0.30
    porcentaje_renta_fija = 0.70
    mensaje_distribucion = "**Perfil de Inversión:** Conservador (30% Renta Variable, 70% Renta Fija). **Fundamento:** Al tener un horizonte de inversión a corto plazo, se busca proteger el capital y obtener rendimientos moderados."

# Calcula el monto total acumulado al final del período
monto_total_acumulado = calculate_future_value(aportación_inicial, aportación_mensual_requerida * 12, tasa_rendimiento_anual, años_hasta_jubilación)

fundamento = f"**Fundamento:** Al invertir una parte en renta variable (IPC) y otra en renta fija (CETTES), se busca diversificar la inversión y balancear el riesgo y la rentabilidad."


# Mostrar mensaje de distribución con estilo
st.markdown(f"""
    <style>
        .distribution-message {{
            font-family: 'Verdana', sans-serif;
            font-size: 18px;
            color: #dc3545; /* Color rojo */
            text-align: justify; /* Justificar el texto */
        }}
    </style>
    <div class="distribution-message">{mensaje_distribucion}</div>
""", unsafe_allow_html=True)

# Mostrar fundamento con estilo
st.markdown(f"""
    <style>
        .fundamento {{
            font-family: 'Georgia', serif;
            font-size: 16px;
            color: #6c757d; /* Color gris */
            text-align: justify;
        }}
    </style>
    <div class="fundamento">{fundamento}</div>
""", unsafe_allow_html=True)


# Mostrar monto total acumulado con estilo
if años_hasta_jubilación >= 0:
    st.markdown(f"""
        <style>
            .total-amount {{
                font-family: 'Impact', sans-serif;
                font-size: 36px;
                color: #ffc107; /* Color amarillo */
                text-align: center;
            }}
        </style>
        <div class="total-amount">Con esta distribución, el monto total acumulado al final del período será de aproximadamente: **{monto_total_acumulado:.2f}**</div>
    """, unsafe_allow_html=True)
    st.write(f"Recuerda que esta simulación utiliza las siguientes tasas: Renta Variable: {tasa_renta_variable * 100:.2f}% anual (IPC), Renta Fija: {tasa_renta_fija * 100:.2f}% anual (CETTES).")

2025-04-08 02:31:33.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.735 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 02:31:33.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar